In [9]:
import pandas as pd
import numpy as np
# import nltk
# import stop_words
from nltk.corpus import stopwords
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score, StratifiedKFold, ParameterGrid
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import date
# import fastnumbers
import re
# from fastnumbers import isfloat, isint
import eli5
from eli5.lime import TextExplainer

In [39]:
from nltk.tokenize import word_tokenize

In [12]:
from nltk.stem.porter import PorterStemmer

In [13]:
ps = PorterStemmer()

In [38]:
ps.stem('каретой')

'каретой'

In [26]:
from nltk.stem.wordnet import WordNetLemmatizer

In [44]:
wnl = WordNetLemmatizer()

In [47]:
wnl.lemmatize('беспокоили')

'беспокоили'

In [53]:
import pymorphy2

In [216]:
morph = pymorphy2.MorphAnalyzer()

In [223]:
morph.parse('разобраться')[0].normal_form

'разобраться'

In [287]:
data = pd.read_csv('03online.csv', encoding='utf-8')

In [288]:
data.head()

,doctor,eng_doctor,person_age,person_gender,person_name,question,question_title,doctor_answer
0,Гинеколог,ginekologija,38,Женский,"Ольга, Ростов-на-Дону","Здравствуйте, помогите, пожалуйста, разобратьс...",Расшифровка результата мазка и цитологии,NaN
1,Гинеколог,ginekologija,20,Женский,"алена, южно сахалинск","Здравствуйте, у меня сложилась такая проблема ...",Задержка месячных и обильные выделения,NaN
2,Гинеколог,ginekologija,31,Женский,Анастасия,"Здравствуйте, я уже 5 месяцев принимаю джес. П...",Препарат джес и узи органов малого таза,NaN
3,Гинеколог,ginekologija,23,Женский,"Лина, Москва","Здравствуйте, такая проблема, последние месячн...",Беременность или ПМС?,NaN
4,Гинеколог,ginekologija,16,Женский,"Сабрина, Россия, Башкортостан,Уфа","Здравствуйте, мне 16 лет. Я не девственница с ...",Нужно ли идти к гинекологу и венерологу?,NaN


In [65]:
#data['question'].astype(str).apply(lambda x: x.lower())
#data['question_title'].astype(str).apply(lambda x: [ps.stem(i) for i in word_tokenize(x)])[0]
data['question'].astype(str).apply(lambda x: [morph.parse(i)[0].normal_form for i in word_tokenize(x)])[0]
#data['question'].astype(str).apply(lambda x: " ".join([ps.stem(i) for i in word_tokenize(x)]))[0]

['здравствовать',
 ',',
 'помочь',
 ',',
 'пожалуйста',
 ',',
 'разобраться',
 '.',
 'обращаться',
 'очно',
 'к',
 'гинеколог',
 '-',
 'беспокоить',
 'вернуться',
 'признак',
 'два',
 'год',
 'назад',
 'пролечить',
 'молочница',
 '.',
 'в',
 'микроскопия',
 'отделять',
 'на',
 'флор',
 'и',
 'клеточный',
 'состав',
 '-',
 'лейкоцит',
 '-',
 '18-20',
 'в',
 'слизь',
 'значительный',
 ',',
 'эпителий',
 '-',
 '4-6',
 ',',
 'слизь',
 '-',
 'значительный',
 ',',
 'кокк',
 '-',
 'обнаружить',
 ',',
 'незначительно',
 ';',
 'палочка',
 '-',
 'обнаружить',
 'значительно',
 ';',
 'дрожь',
 '.',
 'клетка',
 '-',
 'обнаружить',
 'значительно',
 ';',
 'мицелий',
 '-',
 'обнаружить',
 ',',
 'значительно',
 '.',
 'остальной',
 'параметр',
 '-',
 'не',
 'обнаружить',
 '.',
 'цитологический',
 'исследование',
 '-',
 'качество',
 'препарат',
 'адекватный',
 '.',
 'цитограмм',
 '-',
 'в',
 'получить',
 'материал',
 'обнаружить',
 'клетка',
 'плоский',
 'эпителий',
 'поверхностный',
 ',',
 'промежуточны

In [101]:
qq=data['question'][0]
qq=re.sub(u"[^а-яА-Я0-9.,\-\s]|(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)|()[,.-]", "", qq.lower()).split()
qq=[morph.parse(i)[0].normal_form for i in qq]
print(qq)

['здравствовать', 'помочь', 'пожалуйста', 'разобраться', 'обращаться', 'очно', 'к', 'гинеколог', 'беспокоить', 'вернуться', 'признак', 'два', 'год', 'назад', 'пролечить', 'молочница', 'в', 'микроскопия', 'отделять', 'на', 'флор', 'и', 'клеточный', 'состав', 'лейкоцит', '1820', 'в', 'слизь', 'значительный', 'эпителий', '46', 'слизь', 'значительный', 'кокк', 'обнаружить', 'незначительно', 'палочка', 'обнаружить', 'значительно', 'дрожь', 'клетка', 'обнаружить', 'значительно', 'мицелий', 'обнаружить', 'значительно', 'остальной', 'параметр', 'не', 'обнаружить', 'цитологический', 'исследование', 'качество', 'препарат', 'адекватный', 'цитограмм', 'в', 'получить', 'материал', 'обнаружить', 'клетка', 'плоский', 'эпителий', 'поверхностный', 'промежуточный', 'слой', 'и', 'цилиндрический', 'эпителий', 'умеренно', 'выразить', 'лейкоцитарный', 'инфильтрация', 'клетка', 'с', 'признак', 'злокачественность', 'не', 'выявить', 'флор', 'полиморфнопалочковый', 'врач', 'обещать', 'позвонить', 'по', 'расшифр

In [177]:
data['question'][0]

'Здравствуйте, помогите, пожалуйста, разобраться. Обращалась очно к гинекологу - беспокоили вернувшиеся признаки два года назад пролеченной молочницы. В микроскопии отделяемого на флору и клеточный состав - лейкоциты - 18-20 в слизи значительное, эпителий - 4-6, слизь - значительное, кокки - обнаружены, незначительно; палочки - обнаружены значительно; дрожь. Клетки - обнаружены значительно; мицелий - обнаружен, значительно. Остальные параметры - не обнаружены. Цитологическое исследование - качество препарата адекватное. Цитограмма - в полученном материале обнаружены клетки плоского эпителия поверхностного, промежуточного слоев и цилиндрического эпителия. Умеренно выраженная лейкоцитарная инфильтрация. Клетки с признаки злокачественности не выявлены. Флора - полиморфно-палочковая. Врач обещала позвонить по расшифровке результатов, но забыла про меня. Зуд возобновился, а ей позвонить я смогу только через три дня. Поможете разобраться?'

In [272]:
text = data['question'][0]
#clearText = re.sub(u"[^а-яА-Я0-9.,\-\s]", "", text.lower())
#clearText = re.sub(u"[.,\-\s]{3,}", " ", clearText)
someText = re.sub(u"[,.-]|[0-9]"," ",text.lower()).split()
#[morph.parse(i)[0].normal_form for i in someText]
someText

['здравствуйте',
 'помогите',
 'пожалуйста',
 'разобраться',
 'обращалась',
 'очно',
 'к',
 'гинекологу',
 'беспокоили',
 'вернувшиеся',
 'признаки',
 'два',
 'года',
 'назад',
 'пролеченной',
 'молочницы',
 'в',
 'микроскопии',
 'отделяемого',
 'на',
 'флору',
 'и',
 'клеточный',
 'состав',
 'лейкоциты',
 'в',
 'слизи',
 'значительное',
 'эпителий',
 'слизь',
 'значительное',
 'кокки',
 'обнаружены',
 'незначительно;',
 'палочки',
 'обнаружены',
 'значительно;',
 'дрожь',
 'клетки',
 'обнаружены',
 'значительно;',
 'мицелий',
 'обнаружен',
 'значительно',
 'остальные',
 'параметры',
 'не',
 'обнаружены',
 'цитологическое',
 'исследование',
 'качество',
 'препарата',
 'адекватное',
 'цитограмма',
 'в',
 'полученном',
 'материале',
 'обнаружены',
 'клетки',
 'плоского',
 'эпителия',
 'поверхностного',
 'промежуточного',
 'слоев',
 'и',
 'цилиндрического',
 'эпителия',
 'умеренно',
 'выраженная',
 'лейкоцитарная',
 'инфильтрация',
 'клетки',
 'с',
 'признаки',
 'злокачественности',
 'не'

In [82]:
print(re.split('\W|\d',text))

['текст', '', 'нормальный', 'принормальный', '', 'русский', '', 's', '', 'musorom', '', '', 'английским', '']


In [89]:
print(re.sub(u"[^а-яА-Я0-9.,\-\s]|(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)|()[,.-]", "", text).split())

['текст', 'нормальныйпринормальный', 'русский', 'английским']


In [285]:
data = pd.read_csv('03online.csv', encoding='utf-8')

data['question'] = data['question'].astype(str)\
    .apply(lambda x: re.sub(u'[,.-]|[0-9]', ' ', x.lower()))\
    .apply(lambda x: re.sub('\s+', ' ', x))\
    .apply(lambda x: [morph.parse(i)[0].normal_form for i in x.split()])
    
data.head()

,doctor,eng_doctor,person_age,person_gender,person_name,question,question_title,doctor_answer
0,Гинеколог,ginekologija,38,Женский,"Ольга, Ростов-на-Дону","[здравствовать, помочь, пожалуйста, разобратьс...",Расшифровка результата мазка и цитологии,NaN
1,Гинеколог,ginekologija,20,Женский,"алена, южно сахалинск","[здравствовать, у, я, сложиться, такой, пробле...",Задержка месячных и обильные выделения,NaN
2,Гинеколог,ginekologija,31,Женский,Анастасия,"[здравствовать, я, уже, месяц, принимать, джес...",Препарат джес и узи органов малого таза,NaN
3,Гинеколог,ginekologija,23,Женский,"Лина, Москва","[здравствовать, такой, проблема, последний, ме...",Беременность или ПМС?,NaN
4,Гинеколог,ginekologija,16,Женский,"Сабрина, Россия, Башкортостан,Уфа","[здравствовать, я, год, я, не, девственница, с...",Нужно ли идти к гинекологу и венерологу?,NaN


In [334]:
data = pd.read_csv('03online-2.csv', encoding='utf-8')
data.head(2)

,age,answer,answer_date,doctor,doctor_en,doctor_name,gender,name,question,title
0,26,NaN,NaN,Маммолог,NaN,NaN,Женский,"алина, москва","Здравствуйте, мне 26 лет. Я беременна до берем...",В молочной железе как будто есть сеточка
1,21,NaN,NaN,Маммолог,NaN,NaN,Женский,Ирина,"Здравствуйте, проходила плановое обследование ...",Рекомендовано через 3 месяца контрольное УЗИ м...


In [335]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59058 entries, 0 to 59057
Data columns (total 10 columns):
age            59019 non-null object
answer         56761 non-null object
answer_date    56761 non-null object
doctor         59058 non-null object
doctor_en      56722 non-null object
doctor_name    39 non-null object
gender         59058 non-null object
name           59058 non-null object
question       59058 non-null object
title          59058 non-null object
dtypes: object(10)
memory usage: 4.5+ MB


In [286]:
%%time
#data['question'] = data['question'].astype(str).apply(lambda x: x.lower())
data = pd.read_csv('03online.csv', encoding='utf-8')

data['question'] = data['question'].astype(str)\
    #.apply(lambda x: re.sub(u'[^а-яА-Я0-9.,\-\s]|(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)', ' ', x.lower()) ) \
    .apply(lambda x: re.sub(u'[,.-]|[0-9]', ' ', x.lower()))\
    .apply(lambda x: re.sub(u'\s+', ' ', x))\
    .apply(lambda x: [morph.parse(i)[0].normal_form for i in x.split()])

IndentationError: unexpected indent (<unknown>, line 5)

In [336]:
data['question'] = data['question'].astype(str)\
    .apply(lambda x: re.sub(u'[,.-]|[0-9]', ' ', x.lower()))\
    .apply(lambda x: re.sub('\s+', ' ', x))#\
    #.apply(lambda x: [morph.parse(i)[0].normal_form for i in x.split()])

data.head()

,age,answer,answer_date,doctor,doctor_en,doctor_name,gender,name,question,title
0,26,NaN,NaN,Маммолог,NaN,NaN,Женский,"алина, москва",здравствуйте мне лет я беременна до беременнос...,В молочной железе как будто есть сеточка
1,21,NaN,NaN,Маммолог,NaN,NaN,Женский,Ирина,здравствуйте проходила плановое обследование у...,Рекомендовано через 3 месяца контрольное УЗИ м...
2,18,NaN,NaN,Маммолог,NaN,NaN,Женский,Мария,добрый день я поехала в деревню решила поглади...,Лошадь укусила за грудь. Теперь в груди затвер...
3,25,NaN,NaN,Маммолог,NaN,NaN,Женский,"Юлия, Орел",здравствуйте на узи нашли фибро аденому в этот...,Нашли фибро аденому. Пью мндиану. Можно ли?
4,45,NaN,NaN,Маммолог,NaN,NaN,Женский,"Оксана, Москва",здравствуйте алексей александрович страдаю фиб...,Кальцинаты


In [357]:
data['title'] = data['title'].astype(str)\
    .apply(lambda x: re.sub(u'[,.-]|[0-9]', ' ', x.lower()))\
    .apply(lambda x: re.sub('\s+', ' ', x))#\
    #.apply(lambda x: [morph.parse(i)[0].normal_form for i in x.split()])

data.head()

,age,answer,answer_date,doctor,doctor_en,doctor_name,gender,name,question,title
0,26,NaN,NaN,Маммолог,NaN,NaN,Женский,"алина, москва",здравствуйте мне лет я беременна до беременнос...,в молочной железе как будто есть сеточка
1,21,NaN,NaN,Маммолог,NaN,NaN,Женский,Ирина,здравствуйте проходила плановое обследование у...,рекомендовано через месяца контрольное узи м/ж...
2,18,NaN,NaN,Маммолог,NaN,NaN,Женский,Мария,добрый день я поехала в деревню решила поглади...,лошадь укусила за грудь теперь в груди затверд...
3,25,NaN,NaN,Маммолог,NaN,NaN,Женский,"Юлия, Орел",здравствуйте на узи нашли фибро аденому в этот...,нашли фибро аденому пью мндиану можно ли?
4,45,NaN,NaN,Маммолог,NaN,NaN,Женский,"Оксана, Москва",здравствуйте алексей александрович страдаю фиб...,кальцинаты


In [342]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['doctor', 'doctor_en'], axis=1), data['doctor'], 
                                                    test_size=0.2, random_state=42)

In [344]:
X_train[:3]

,age,answer,answer_date,doctor_name,gender,name,question,title
45985,21,В вашем возрасте уже пора знать элементарную а...,2015-06-18 22:41,NaN,Женский,"Марина, Москва",здравствуйте был незащищенный оральный секс со...,Незащищенный оральный секс
56772,22,Здравствуйте. Вам нужно пройти обследование до...,2016-12-11 21:24,NaN,Женский,"Алия, Сибай",здравствуйте доктор! у меня такие симптомы чув...,Чувство мурашек на лбу
5304,21,Здравствуйте! Во время лактации не рекомендует...,2014-06-05 08:59,NaN,Женский,Анастасия,добрый день подскажите можно ли при лактации х...,Солярий при грудном вскармливании


In [345]:
y_test.value_counts()[:5]

Невропатолог         1655
Терапевт             1621
Педиатр              1607
Гинеколог            1552
Сексолог-Андролог    1403
Name: doctor, dtype: int64

In [346]:
y_train.value_counts()[:5]

Гинеколог            6468
Педиатр              6401
Терапевт             6385
Невропатолог         6350
Сексолог-Андролог    5628
Name: doctor, dtype: int64

In [347]:
data.shape

(59058, 10)

In [348]:
X_train.shape

(47246, 8)

In [349]:
X_test.shape

(11812, 8)

In [350]:
y_train.shape

(47246,)

In [351]:
y_test.shape

(11812,)

In [352]:
class TextPreprocessing(BaseEstimator, TransformerMixin):
    def __init__(self, stopwords):
        self.stopwords = stopwords
    def fit(self, X, y):
        from numpy import shape, argsort, ceil
        X_pos, X_neg = X[y==1], X[y==0]
        X_mean = X.mean(axis=0)
        X_pos_mean, X_neg_mean = X_pos.mean(axis=0), X_neg.mean(axis=0)
        deno = (1.0/(shape(X_pos)[0]-1))*X_pos.var(axis=0) + (1.0/(shape(X_neg)[0]-1))*X_neg.var(axis=0)
        num = (X_pos_mean - X_mean)**2 + (X_neg_mean - X_mean)**2
        F = num/deno
        sort_F = argsort(F)[::-1]
        n_feature = (float(self.percentile)/100)*shape(X)[1]
        self.ind_feature = sort_F[:ceil(n_feature)]
        return self
    def transform(self, x):
        return x[self.ind_feature,:]

In [353]:
# nltk.download()

In [354]:
classifier = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer = 'char', max_features = 1000000, lowercase = True, 
                                   stop_words = stopwords.words('russian'), ngram_range = (1, 4))),
    #('clf', OneVsRestClassifier(LogisticRegression(),n_jobs=-1)),
    ('lsvc',LinearSVC())])

In [362]:
%%time
classifier.fit(X_train['question'], y_train)

CPU times: user 2min 19s, sys: 4.46 s, total: 2min 24s
Wall time: 2min 26s


Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000000, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [363]:
predicted = classifier.predict(X_test['question'])
accuracy_score(predicted, y_test)

0.82450050795800878

In [369]:
classifier.predict(["болит живот и знобит"])

array(['Терапевт'], dtype=object)